In [23]:
import torch.nn as nn
import librosa
import numpy as np
import torch
import torch.utils.data
import matplotlib.pyplot as plt
import IPython.display as ipd
import os
from torch.autograd import Variable

In [24]:
# from google.colab import drive
# drive.mount('/content/drive/')

# os.chdir('/content/drive/My Drive/Colab Notebooks')

In [25]:
training_folder = 'tr'
validation_folder = 'v'
test_folder = 'te'
file_extension = '.wav'
train_dirty_file_path = 'numpy_files/file_names/train/train_dirty_files.npy'
train_speech_file_path = 'numpy_files/file_names/train/train_speech_files.npy'
train_noise_file_path = 'numpy_files/file_names/train/train_noise_files.npy'

train_librosa_n_list_path = 'numpy_files/librosa_data/train/train_n_list.npy'
train_librosa_s_list_path = 'numpy_files/librosa_data/train/train_s_list.npy'
train_librosa_x_list_path = 'numpy_files/librosa_data/train/train_x_list.npy'

train_librosa_complex_list_path = 'numpy_files/librosa_data/train/train_complex_list.npy'

test_dirty_file_path = 'numpy_files/file_names/test/test_dirty_files.npy'
test_librosa_x_list_path = 'numpy_files/librosa_data/test/test_x_list.npy'
test_librosa_complex_list_path = 'numpy_files/librosa_data/test/test_complex_list.npy'
dimension = 513
file_count_to_denoise = 1200
load_existing_files = True
test_files_count = 10
epochs = 1
sample_rate = 16000
save_files = False

In [26]:
def get_file_number(number):
    return "0" * (4 - len(number)) + number + file_extension

In [27]:
def file_exists(file):
  return os.path.isfile(file)

In [28]:
def get_files(folder, test=False):
  dirty_files = []
  speech_files = []
  noise_files = []
  for i in range(file_count_to_denoise):
      dirty_files.append(folder+'/' + folder + 'x'  + get_file_number(str(i)))
      if not test:
        speech_files.append(folder+'/' + folder +'s' + get_file_number(str(i)))
        noise_files.append(folder+ '/' + folder + 'n' + get_file_number(str(i)))
  if not test:
    return np.array(dirty_files), np.array(speech_files), np.array(noise_files)
  return np.array(dirty_files)

In [29]:
if load_existing_files:
  print('loading existing files')
  train_dirty_files = np.load(train_dirty_file_path)
  train_speech_files = np.load(train_speech_file_path)
  train_noise_files = np.load(train_noise_file_path)

else:
  train_dirty_files, train_speech_files, train_noise_files = get_files(training_folder)

train_dirty_files = train_dirty_files[:file_count_to_denoise].reshape(file_count_to_denoise, 1)
train_speech_files = train_speech_files[:file_count_to_denoise].reshape(file_count_to_denoise,1)
train_noise_files = train_noise_files[:file_count_to_denoise].reshape(file_count_to_denoise,1)
  
# if save_files:
#   np.save(train_dirty_file_path, train_dirty_files)
#   np.save(train_speech_file_path, train_speech_files)
#   np.save(train_noise_file_path, train_noise_files)
 
# test_dirty_files, test_speech_files, test_noise_files = get_files(test_folder)


loading existing files


In [30]:
print(train_dirty_files.shape)
print(train_speech_files.shape)

(1200, 1)
(1200, 1)


In [31]:
def get_processed_lists(dirty_files, speech_files, noise_files, test=False):
  x_list = np.zeros(shape=(dimension,1))
  x_complex = np.zeros(shape=(dimension,1))
  s_list = np.zeros(shape=(dimension,1))
  n_list = np.zeros(shape=(dimension,1))
  for i in range(file_count_to_denoise):
      print("Running for tile", str(i))
  #     for dirty files
      x, srx=librosa.load(dirty_files[i][0], sr=None)
      X=librosa.stft(x, n_fft=1024, hop_length=512)
  #     for speech files
    
      s, srs=librosa.load(speech_files[i][0], sr=None)
      S=librosa.stft(s, n_fft=1024, hop_length=512)
    #     for noise files
      n, srn=librosa.load(noise_files[i][0], sr=None)
      N=librosa.stft(n, n_fft=1024, hop_length=512)
      
      
      x_abs = np.abs(X)
      s_abs = np.abs(S)
      n_abs = np.abs(N)
      
      if x_list.shape[1] == 1:
          x_list = x_abs
          x_complex = X
          s_list = s_abs
          n_list = n_abs
      else:  
          x_list = np.append(x_list, x_abs,1)
          x_complex = np.append(x_complex, X, 1)
          s_list = np.append(s_list, s_abs,1)
          n_list = np.append(n_list, n_abs,1)

  x_list = x_list.T
  x_complex = x_complex.T
  
  n_list = n_list.T
  s_list = s_list.T
   

  return s_list,n_list,x_list, x_complex

In [32]:
def get_processed_list_test(dirty_files):
  x_list = []
  x_complex = []
    
  for i in range(test_files_count):
      print("Running for tile", str(i))
  #     for dirty files
      x, srx=librosa.load(dirty_files[i][0], sr=None)
      X=librosa.stft(x, n_fft=1024, hop_length=512)
      
      
      x_abs = np.abs(X)
 
      x_list.append(x_abs)
      x_complex.append(X)

  return x_list, x_complex

In [33]:
if load_existing_files:
  train_s_list = np.load(train_librosa_s_list_path)
  train_n_list = np.load(train_librosa_n_list_path)
  train_x_list = np.load(train_librosa_x_list_path)
  train_x_complex = np.load(train_librosa_complex_list_path)
else:
  train_s_list, train_n_list, train_x_list, train_x_complex = get_processed_lists(train_dirty_files, train_speech_files, train_noise_files)

In [34]:
# if save_files:
#   np.save(train_librosa_s_list_path, train_s_list)
#   np.save(train_librosa_n_list_path, train_n_list)
#   np.save(train_librosa_x_list_path, train_x_list)
#   np.save(train_librosa_complex_list_path, train_x_complex)

In [35]:
print(train_s_list.shape)
print(train_x_list.shape)
print(train_x_complex.shape)

(118550, 513)
(118550, 513)
(118550, 513)


In [36]:
def get_IBM(n_list, s_list):
  length = len(s_list)
  ibm_list = np.zeros(shape=(length,513))
  for i in range(length):
    ibm = np.zeros(len(s_list[i]))
    ibm[s_list[i] > n_list[i]] = 1
    ibm_list[i] = ibm
  return ibm_list 
  

In [37]:

ibm = get_IBM(train_n_list, train_s_list)

In [38]:
ibm = np.expand_dims(ibm, 2)
train_x_list = np.expand_dims(train_x_list, 2)

In [39]:
# BATCH = 10
# train_loader = torch.utils.data.DataLoader(train_x_list, batch_size=BATCH)
# test_loader = torch.utils.data.DataLoader(ibm, batch_size=BATCH)

In [40]:
#Ref : https://nipunbatra.github.io/blog/2018/denoising.html
class GRU(nn.Module):
    
#     def step(self, pred):
#         greater = pred >=0.5
#         less = pred < 0.5
#         pred[greater] = 1
#         pred[less] = 0
#         return pred
  
    def __init__(self, input_size, hidden_size):
        super(GRU, self).__init__()
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
        self.linear = nn.Linear(hidden_size, 1)
        self.act = torch.sigmoid
        
    def forward(self, x):
        pred, hidden = self.rnn(x, None)
        pred = self.act(self.linear(pred)).view(pred.data.shape[0], -1, 1)
        
#         greater = pred >=0.5
#         less = pred < 0.5
#         pred[greater] = 1
#         pred[less] = 0
        return pred
model = GRU(1, 1).cuda()
print(model)

RuntimeError: CUDA error: unspecified launch failure

In [43]:
loss_function= nn.MSELoss()
para = model.parameters()
optimizer = torch.optim.Adam(params=para, lr=0.001)
loss_list = []
x_train = train_x_list
y_train = ibm
for i in range(epochs):            
    model_input = Variable(torch.Tensor(x_train.reshape((x_train.shape[0], -1, 1))), requires_grad=True).cuda()
    target = Variable(torch.Tensor(y_train.reshape((y_train.shape[0], -1, 1))) ).cuda()
    optimizer.zero_grad()
    network_output  = model(model_input)
    print(network_output.shape)
    print(target.shape)
    loss = loss_function(network_output , target)
    loss.backward()
    optimizer.step()
    numpy_loss = loss.data.cpu().numpy()
    print("Epoch" + str(i) , "Loss" + str(numpy_loss))
    loss_list.append(numpy_loss)
plt.plot(range(epochs), loss_list)
plt.show()      

tensor([[[0.3853],
         [0.0942],
         [0.0525],
         ...,
         [0.1368],
         [0.0008],
         [0.0761]],

        [[0.0605],
         [0.2775],
         [0.6325],
         ...,
         [0.0603],
         [0.0517],
         [0.0866]],

        [[0.0117],
         [0.2630],
         [0.5427],
         ...,
         [0.0914],
         [0.0500],
         [0.0138]],

        ...,

        [[0.3467],
         [0.2731],
         [0.1912],
         ...,
         [0.1886],
         [0.0834],
         [0.1052]],

        [[0.1575],
         [0.2372],
         [0.0755],
         ...,
         [0.0502],
         [0.0623],
         [0.0383]],

        [[0.2707],
         [0.1212],
         [0.0507],
         ...,
         [0.0974],
         [0.0742],
         [0.0487]]], requires_grad=True)


RuntimeError: CUDA error: unspecified launch failure

In [ ]:
# validation_dirty_files, validation_speech_files, validation_noise_files = get_files(validation_folder)
if load_existing_files:
  test_dirty_files = np.load(test_dirty_file_path)
else:
  test_dirty_files = get_files(test_folder, True)


In [ ]:
test_dirty_files = test_dirty_files.reshape(test_dirty_files.shape[0], 1)

In [ ]:
print(test_dirty_files.shape)
# if save_files:
#   np.save(test_dirty_file_path, test_dirty_files)

In [ ]:
test_dirty_files.shape

**--------------------------------------------------------------------------------------------------Testing Begins-----------------------------------------------------**

In [ ]:
# validation_s_list, validation_n_list, validation_x_list = get_processed_lists(validation_dirty_files, validation_speech_files, validation_noise_files)
# if load_existing_files:
#   test_x_list = np.load(test_librosa_x_list_path)
#   test_x_complex = np.load(test_librosa_complex_list_path)
# else:
test_x_list, test_x_complex = get_processed_list_test(test_dirty_files)
x_list = test_x_list

In [ ]:
len(test_x_list)

In [ ]:
test_x_list[11].shape

In [ ]:
np.array(test_x_list)

In [ ]:
# if save_files:
#   np.save(test_librosa_x_list_path, test_x_list)
#   np.save(test_librosa_complex_list_path, test_x_complex)

In [ ]:
x_list[0].shape

In [ ]:
x_list[0].reshape(x_list[0].shape[0],x_list[0].shape[1], 1).shape

In [ ]:
output_istft = []
for i in range(len(x_list)):
    model_input = Variable(torch.Tensor(x_list[i].reshape(x_list[i].shape[0],x_list[i].shape[1], 1)))
    prediction = model(model_input.cuda())
    y = prediction.detach().cpu().numpy()
    print(y.shape)
#     print(test_x_complex.shape)
#     print(y.shape)
    result = np.multiply(test_x_complex[i],y.reshape(y.shape[0],y.shape[1]))
    # spec_istft = librosa.istft(spec, hop_length=512)
    result_istft = librosa.istft(result, hop_length=512)
    result_istft = result_istft.reshape(result_istft.shape[0],1)
    output_istft.append(result_istft)
#     print(result.shape)

In [ ]:
# prediction = 1 * (prediction > 0.5)

In [ ]:
# m_input =  model_input.detach().numpy()

# spec = (m_input.reshape(dimension,result.shape[0]) / np.abs(m_input).reshape(dimension,result.shape[0])) * result.T

In [ ]:
# print(srs)
# print(srn)
# print(srx)
# librosa.output.write_wav('validation2.wav', spec_istft, srs)
librosa.output.write_wav('test_result.wav', output_istft[0], sample_rate)

In [ ]:
# ipd.Audio('validation2.wav')


In [ ]:
ipd.Audio('test_result.wav')

In [ ]:
# s = test_x_complex
# print(s.shape)
# y = y.reshape(result.shape[0], dimension)
# print(y.shape)
# print(np.max(y))

In [ ]:
# print(10*np.log10(np.sum(np.square(test_x_complex)/(np.sum(np.square(np.subtract(test_x_complex ,y)))))))